In [240]:
import altair as alt
import pandas as pd
from vega_datasets import data
from geopy.geocoders import Nominatim

In [241]:
geolocator = Nominatim(user_agent="vis_cw")

In [242]:
def is_country(lat, long, geolocator, country):
    string = f'{lat}, {long}'
    locationString = geolocator.reverse("41.285487, 15.917172")
    locationArr = locationString.address.split(', ')
    if len(locationArr) > 0:
        responseCountry = locationArr[-1]
        if type(country) == str:
            return responseCountry == country
        elif type(country) == list:
            for x in country:
                if x == responseCountry:
                    return True
    return False

In [243]:
mp = data.world_110m.url

In [244]:
df = pd.read_csv('pleiades.csv')
df = df[df.hasConnectionsWith.notnull()]
# df['isItaly'] = df.apply(lambda row: is_country(row.reprLat, row.reprLong, geolocator, ['Italia', 'Italy']), axis=1)

In [245]:
connections = []
for row in zip(df['id'], df['hasConnectionsWith']):
    spl = row[1].split(',')
    for x in spl:
        connections.append([row[0], x])

connectionsDf = pd.DataFrame(connections, columns=["origin", "connection"])
print(connectionsDf.head())

   origin connection
0  265876     266518
1  265877     266018
2  265884  482947334
3  265884  482947335
4  265884     265883


In [246]:
print(connectionsDf[connectionsDf.origin == 344448])

      origin connection
4263  344448  133229241
4264  344448  716466643
4265  344448  960592954
4266  344448  358782736
4267  344448  371761216
4268  344448  165453294
4269  344448     344573


In [238]:
alt.data_transformers.disable_max_rows()

select_city = alt.selection_single(
    on="mouseover", nearest=True, fields=["origin"], empty="none"
)

lookup_data = alt.LookupData(
    df, key="id", fields=["title", "reprLat", "reprLong"]
)

mp = data.world_110m.url
source = alt.topo_feature(mp, 'countries')

globe = alt.Chart(alt.sphere()).mark_geoshape(fill='lightblue', opacity=0.7)
meridian = alt.Chart(alt.graticule()).mark_geoshape(stroke='white', strokeWidth=0.3, opacity=0.8)

background = alt.Chart(source).mark_geoshape(
    fill="lightgray",
    stroke="white"
)

connections = alt.Chart(connectionsDf).mark_rule(opacity=0.35).encode(
    latitude="reprLat:Q",
    longitude="reprLong:Q",
    latitude2="reprLat2:Q",
    longitude2="reprLong2:Q"
).transform_lookup(
    lookup="origin",
    from_=lookup_data
).transform_lookup(
    lookup="connection",
    from_=lookup_data,
    as_=["title", "reprLat2", "reprLong2"]
).transform_filter(
    select_city
)

points = alt.Chart(connectionsDf).mark_circle().encode(
    latitude="reprLat:Q",
    longitude="reprLong:Q",
    tooltip=["title:N", "origin:N"]
).transform_lookup(
    lookup="origin",
    from_=lookup_data
).add_selection(
    select_city
)

alt.layer(
    globe,
    meridian,
    background,
    points,
    connections
).properties(
    width=750,
    height=500
).project(
    "mercator",
    scale=1000,
    center=[14, 40],
#     clipExtent=[[0, 0], [650, 500]],
).configure_view(stroke=None)

alt.LayerChart(...)